# Pytorch 基本用法（二）
本节介绍 Pytorch 内部的自动求导机制。

下面可能会把求导和求梯度两个词有一些混用，但是本质上是同一件事。

## 计算图
很多地方在讲解反向传播的时候都用**计算图**来演示传播的过程。

![](pytorch_notes_assets/1.png)

Pytorch 的自动微分机制差不多就是基于计算图实现的。每一个圆点代表 `Tensor` 对象，方点则代表 `Function` 对象。


## 求梯度的开关
Pytorch 非常厉害的一点就在于可以自动计算梯度。

之前介绍的 `Tensor` 是 Pytorch 的核心类，如果将其属性 `.requires_grad` 设置为 `True`，它将开始追踪（或者说，记录）在其上的所有操作。使用 `.requires_grad_()` 方法也可以改变这个属性。

一个张量是否被计算梯度和它所处的环境有关。如果一个输入需要计算梯度，那么输出就要计算梯度；反之如果所有输入都不需要梯度，那么输出就不需要梯度。

**注意：如果处于追踪状态，那么一个计算图只有叶子节点可以停止被追踪，非叶节点则不可以。**

In [29]:
import torch
a = torch.ones((2, 2), requires_grad=True)
b = torch.eye(2)
c = a + b
print(c.requires_grad)
a.requires_grad_(False)
print(a.requires_grad)
# c.requires_grad_(False)
"""
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
<ipython-input-26-8098baefffea> in <module>
      6 a.requires_grad_(False)
      7 print(a.requires_grad)
----> 8 c.requires_grad_(False)

RuntimeError: you can only change requires_grad flags of leaf variables. 
If you want to use a computed variable in a subgraph that doesn't require differentiation use var_no_grad = var.detach().
"""
pass

True
False


如果要让一个张量停止被追踪，可以调用 `.detach_()` 方法。如果调用不带下划线的就会返回该张量的一个**浅拷贝**，该拷贝不被记录梯度。

In [16]:
d = c.detach()
d[0, 0] = 111
print(c)

tensor([[111.,   1.],
        [  1.,   2.]], grad_fn=<AddBackward0>)


在某些不需要梯度的环境下（如测试模型时）可以直接使用 `with torch.no_grad()` 将不想被追踪的操作代码块包裹起来。

## 反向传播
每个张量都有一个 `.grad_fn` 属性，该属性即创建该张量的 `Function`, 就是说该张量是不是通过某些运算得到的，若是，则 `grad_fn` 返回一个与这些运算相关的对象，否则是 `None`。

（这里也从某种程度上体现了 Pytorch 的面向对象思想...？）

In [46]:
x = torch.randn(3, 4, requires_grad=True)
y = torch.randn(3, 4)
z = torch.randn(3, 4)
a = x * y
b = a + z
c = torch.sum(b)
print(c.grad_fn)

然后调用 `.backward()` 方法就可以计算梯度，结果将**累积**到 `.grad` 属性中。注意之前用了**累积**一词，这意味着一个 `Tensor` 的梯度默认是累加的，不会自动清空。

In [47]:
c.backward()
print(x.grad)

a = x * y
b = a + z
c = torch.sum(b)
c.backward()
print(x.grad)

tensor([[-0.9333, -0.2024, -0.0907,  1.4957],
        [-0.0571, -0.7191, -0.0418,  1.0434],
        [ 0.4795, -1.1440,  0.3909, -0.9394]])
tensor([[-1.8666, -0.4048, -0.1814,  2.9914],
        [-0.1142, -1.4383, -0.0837,  2.0868],
        [ 0.9589, -2.2880,  0.7819, -1.8788]])


这里输出的就是 $c$ 关于 $x$ 的梯度（偏导数），即 $\frac{\partial c}{\partial x}$。

**注意：这里的 `.backward()` 方法不能被连续调用。**因为 Pytorch 计算梯度使用的是动态图机制，即每一次反向传播都会重新构建计算图，完成传播后销毁。如果要保留计算图需要指定 `retain_graph=True`，不过目前不需要这个。

我们想要每次传播之后清空梯度，就可以使用 `.grad.data.zero_()` 实现梯度的清零。

In [48]:
a = x * y
b = a + z
c = torch.sum(b)
x.grad.data.zero_()
c.backward()
print(x.grad)

tensor([[-0.9333, -0.2024, -0.0907,  1.4957],
        [-0.0571, -0.7191, -0.0418,  1.0434],
        [ 0.4795, -1.1440,  0.3909, -0.9394]])


**注意：这里是已经传播了一次的情况下，最开始 `.grad` 属性为 `None` 时不能这么做。**所以有时候需要特殊判断这种情况。

这里演示的是简单的标量对张量求导，那么张量对张量怎么求呢？这是一个比较复杂的问题，暂且不讨论。